In [50]:
import pandas as pd
import json
import random
import mysql.connector as mysqlpy
import os
from pathlib import Path
import re

## Chargement des datas

In [51]:
def get_db_connection():
    user = os.environ.get('DB_USER')
    password = os.environ.get('DB_PASSWORD')
    host = os.environ.get('DB_HOST')
    port = os.environ.get('DB_PORT')
    database = os.environ.get('DB_DATABASE')
    bdd = mysqlpy.connect(user=user, password=password, host=host, port=port, database=database)
    return bdd

bdd = get_db_connection()
cursor = bdd.cursor()
cursor.execute(f'''SELECT id_CRO, CRO, id_diag, conclusion, position, recommandation, diagnostic FROM CRO 
                   JOIN diagnostics_v2 USING (id_diag)
                   JOIN conclusions USING (id_CRO);''')
CRO = cursor.fetchall()

columns=[i[0] for i in cursor.description]

CRO = pd.DataFrame(CRO, columns= columns)

cursor.execute(f'''SELECT diagnostic, statut, nb_mots FROM diagnostics_v2 WHERE statut = "Train" OR statut = "Test";''')
diag = cursor.fetchall()

columns=[i[0] for i in cursor.description]

diag = pd.DataFrame(diag, columns= columns)

cursor.close()

True

In [52]:
CRO.head(5)

,id_CRO,CRO,id_diag,conclusion,position,recommandation,diagnostic
0,0,Compte rendu d'histopathologie\nDr Valentine B...,175,Le patient Olivie Klein souffre d'un lipome du...,milieu,,lipome
1,1,Compte rendu d'histopathologie\nDr Adrienne Pa...,72,"En conclusion, cette biopsie côlon permet de d...",milieu,,colite éosinophilique
2,4,Compte rendu d'histopathologie\nLe présent rap...,32,Les résultats de l'examen immunohistochimique ...,fin phrase,,cancer indifférencié
3,5,Compte rendu d'histopathologie\nLe présent rap...,61,L'examen histopathologique réalisé à partir de...,fin phrase,,colite auto-immune
4,6,"Compte rendu d'histopathologie\nDr Inès Cohen,...",272,Les résultats de l'examen histopathologique in...,fin phrase,,polypose adénomateuse familiale


In [53]:
lignes_manquantes = CRO[CRO.isna().any(axis=1)]
print(lignes_manquantes)

Empty DataFrame
Columns: [id_CRO, CRO, id_diag, conclusion, position, recommandation, diagnostic]
Index: []


In [54]:
len(CRO)

16542

In [55]:
diag.head(5)

,diagnostic,statut,nb_mots
0,acrodermatite chronique atrophiante,Test,3
1,adénocarcinome,Train,1
2,adénome tubuleux,Train,2
3,adénome tubulovilleux,Test,2
4,adénome villeux,Train,2


In [56]:
def remplace_diag(row):
    diagnostic = row['diagnostic']
    conclusion = row['conclusion']  
    position = row['position']

    if position != 'non trouvé':
        check = re.search(diagnostic, conclusion)
        if check:
            conclusion_sans_diag = conclusion.replace(diagnostic, '[[DIAG]]')
            return conclusion_sans_diag
        else:
            check = re.search(diagnostic.capitalize(), '[[DIAG]]')
            if check:
                conclusion_sans_diag = conclusion.replace(diagnostic.capitalize(), '[[DIAG]]')
    else:
        return conclusion

In [57]:
CRO['conclusion_sans_diag'] = CRO.apply(remplace_diag, axis=1)

In [58]:
CRO.head(5)

,id_CRO,CRO,id_diag,conclusion,position,recommandation,diagnostic,conclusion_sans_diag
0,0,Compte rendu d'histopathologie\nDr Valentine B...,175,Le patient Olivie Klein souffre d'un lipome du...,milieu,,lipome,Le patient Olivie Klein souffre d'un [[DIAG]] ...
1,1,Compte rendu d'histopathologie\nDr Adrienne Pa...,72,"En conclusion, cette biopsie côlon permet de d...",milieu,,colite éosinophilique,"En conclusion, cette biopsie côlon permet de d..."
2,4,Compte rendu d'histopathologie\nLe présent rap...,32,Les résultats de l'examen immunohistochimique ...,fin phrase,,cancer indifférencié,Les résultats de l'examen immunohistochimique ...
3,5,Compte rendu d'histopathologie\nLe présent rap...,61,L'examen histopathologique réalisé à partir de...,fin phrase,,colite auto-immune,L'examen histopathologique réalisé à partir de...
4,6,"Compte rendu d'histopathologie\nDr Inès Cohen,...",272,Les résultats de l'examen histopathologique in...,fin phrase,,polypose adénomateuse familiale,Les résultats de l'examen histopathologique in...


In [59]:
random.seed(0)
CRO = CRO.sample(frac=1).reset_index(drop=True)

In [60]:
conclusions_diag = CRO.query('position != "non trouvé"')
conclusions_diag = conclusions_diag['conclusion_sans_diag'].unique()
conclusions_diag = list(set(conclusions_diag))

conclusions_sans_diag = CRO.query('position == "non trouvé"')
conclusions_sans_diag = conclusions_sans_diag['conclusion_sans_diag'].unique()
conclusions_sans_diag = list(set(conclusions_sans_diag))

recommandations = CRO.query('recommandation != ""')
recommandations = recommandations.recommandation.unique()
recommandations = list(set(recommandations))

conclusions_sans_diag += recommandations

In [61]:
print('Nb de conclusion avec diag', len(conclusions_diag))
print('Nb de conclusion sans diag', len(conclusions_sans_diag))

Nb de conclusion avec diag 15577
Nb de conclusion sans diag 1415


In [62]:
conclusions_diag

['Le diagnostic histopathologique définitif établi après analyse microscopique de la biopsie colorectale est celui de [[DIAG]].',
 "Le diagnostic de [[DIAG]] est étayé par les résultats de l'examen histopathologique. Nous recommandons une surveillance régulière pour monitorer l'évolution de cette maladie et prendre en charge les éventuels relapsus.",
 "En conclusion, cette lésion subcutanée du quadriceps droit du patient était histologiquement compatible avec un [[DIAG]]. Ce type particulier de lipome se caractérise par la présence de cellules lipoblastes, des cellules embryonnaires non différenciées contenant des vacuoles lipidiques. Bien qu'il puisse présenter certaines similitudes morphologiques avec d'autres types de tumeurs, il peut être distingué grâce aux particularités histologiques et immunhistochimiques décrites ci-dessus.",
 "L'analyse histopathologique confirme le diagnostic clinique d'[[DIAG]] chez Mme Noémie Besnard. Nous vous recommandons de poursuivre le traitement pres

In [63]:
index_ref_with_diag = round(0.82 * len(conclusions_diag))
conclusions_train = conclusions_diag[ : index_ref_with_diag]
conclusions_test = conclusions_diag[index_ref_with_diag : ]

print('Nb total :', len(conclusions_diag))
print('Nb en train :', len(conclusions_train))
print('Nb en test :',len(conclusions_test))
print('Somme train + test :',len(conclusions_train) + len(conclusions_test))
print('')
print('Nb total sans diagnostic :', len(conclusions_sans_diag))


Nb total : 15577
Nb en train : 12773
Nb en test : 2804
Somme train + test : 15577

Nb total sans diagnostic : 1415


In [64]:
def generate_data(list_conclusion, diagnostics):
    list_sentence = []
    n = 0
    index = 0
    for conclusion in list_conclusion:
        if conclusion is not None:  
            conclusion = conclusion.replace('[[DIAG]]', diagnostics[n])
            match = re.search(diagnostics[n], conclusion)
            if match:
                entities = [[match.start(), match.end(), 'DIAG']] 
            else:
                print('pas de diagnostic trouvé')
                entities = []
            list_conclusion = [conclusion,{'entities': entities}]
            list_sentence.append(list_conclusion)

            index += 1
            n += 1
            if n >= len(diagnostics):
                n = 0       
        
    return list_sentence

In [65]:
diagnostics_train = diag.query('nb_mots <= 2 and statut == "Train"')['diagnostic']
diagnostics_train = list(diagnostics_train)

diagnostics_test = diag.query('nb_mots <= 2 and statut == "Test"')['diagnostic']
diagnostics_test = list(diagnostics_test)

In [66]:
random.seed(0)
random.shuffle(diagnostics_train)
random.shuffle(diagnostics_test)

In [67]:
total_diags = len(diagnostics_train) + len(diagnostics_test)
print('nb total de diagnostics sélectionnés', total_diags)
print('nb de diagnostics en train:', len(diagnostics_train), 'soit', round(len(diagnostics_train) * 100 / total_diags), '%')
print('nb de diagnostics est:', len(diagnostics_test), 'soit', round(len(diagnostics_test) * 100 / total_diags), '%')

nb total de diagnostics sélectionnés 351
nb de diagnostics en train: 281 soit 80 %
nb de diagnostics est: 70 soit 20 %


In [68]:
print('train avec diag')
train = generate_data(conclusions_train, diagnostics_train)
print('train sans diag')
train_sans_diag = generate_data(conclusions_sans_diag, diagnostics_train)

train avec diag


train sans diag
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de diagnostic trouvé
pas de di

In [69]:
print('test avec diag')
test = generate_data(conclusions_test, diagnostics_test)


test avec diag


In [70]:
def csv_to_json(list_of_sentences: list, file_json : Path) -> str:
    '''
    Génération d'un fichier JSON qui rassemble le texte, les index de début et de fin de chaque mot associé à leur label
    - entrées : 
    - sortie : un fichier JSON
    '''
    # ouverture du fichier JSON
    to_json = json.dumps(list_of_sentences, ensure_ascii=False)

    with open(file_json, 'w', encoding="utf-8") as fichier:
        # écriture du fichier JSON, ajout si existe déjà !
        fichier.write(to_json)
        

In [71]:
index_ref_train_with_diag = round(0.80 * len(train))
index_ref_train_without_diag = round(0.9 *len(train_sans_diag))

train_data = train[: index_ref_train_with_diag] + \
    train_sans_diag[: index_ref_train_without_diag]
valid_data = train[index_ref_train_with_diag:] + \
    train_sans_diag[index_ref_train_without_diag:]
test_data = test

random.seed(0)
random.shuffle(train_data)
random.shuffle(valid_data)
random.shuffle(test_data)

print('génération du fichier json train')
taille_json_train = csv_to_json(train_data, '../generate_model/assets/train.json')
print(f'La data train possède {len(train_data)}')

print('génération du fichier json valid')
taille_json_valid = csv_to_json(valid_data, '../generate_model/assets/dev.json')
print(f'La data valid possède {len(valid_data)}')

print('génération du fichier json test')
taille_json_test = csv_to_json(test_data, '../generate_model/assets/test.json')
print(f'La data test possède {len(test_data)}')

génération du fichier json train
La data train possède 11492
génération du fichier json valid
La data valid possède 2695
génération du fichier json test
La data test possède 2804


In [72]:
print('Pourcentage de textes en valid:', round(len(valid_data) * 100 / (len(valid_data) + len(train_data))),"%")
print('Pourcentage de textes en test:', round(len(test_data) * 100 / (len(valid_data) + len(train_data))),"%")

Pourcentage de textes en valid: 19 %
Pourcentage de textes en test: 20 %
